# Consumer Price Index (CPI) Canada Analysis — Seasonality, Affordability & Market Traps

## PYTHON WORK (Data Preparation + Analysis)

### Import and Load Dataset

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data
dataset = pd.read_csv("C:/Users/DELL/Documents/NOVEMBER/CPI/1810000401_databaseLoadingData.csv")

# Quick look
print(dataset.shape)
dataset.head()


(495, 15)


,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2023-01,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,153.9,NaN,NaN,NaN,1
1,2023-02,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,154.5,NaN,NaN,NaN,1
2,2023-03,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,155.3,NaN,NaN,NaN,1
3,2023-04,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,156.4,NaN,NaN,NaN,1
4,2023-05,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,157.0,NaN,NaN,NaN,1


### Data Cleaning & Transformation

In [9]:
# # Rename columns and strip spaces
dataset = dataset.rename(columns=lambda x: x.strip())

# # Convert REF_DATE to datetime
dataset['REF_DATE'] = pd.to_datetime(dataset['REF_DATE'], errors='coerce')

# # Drop null values in key columns
dataset = dataset.dropna(subset=['REF_DATE', 'VALUE'])

# # Create Month, Year columns
dataset['Month'] = dataset['REF_DATE'].dt.month
dataset['Year'] = dataset['REF_DATE'].dt.year
dataset['Month_Name'] = dataset['REF_DATE'].dt.strftime('%b')

# # Rename product column
for col in dataset.columns:
    if 'product' in col.lower():
        dataset = dataset.rename(columns={col: 'Product'})
        break

# # Filter to Canada (if multiple regions)
if 'GEO' in dataset.columns:
    dataset = dataset[dataset['GEO'] == 'Canada']

print(dataset.head())


    REF_DATE     GEO           DGUID    Product       UOM  UOM_ID  \
0 2023-01-01  Canada  2016A000011124  All-items  2002=100      17   
1 2023-02-01  Canada  2016A000011124  All-items  2002=100      17   
2 2023-03-01  Canada  2016A000011124  All-items  2002=100      17   
3 2023-04-01  Canada  2016A000011124  All-items  2002=100      17   
4 2023-05-01  Canada  2016A000011124  All-items  2002=100      17   

  SCALAR_FACTOR  SCALAR_ID     VECTOR  COORDINATE  VALUE  STATUS  SYMBOL  \
0         units          0  v41690973         2.2  153.9     NaN     NaN   
1         units          0  v41690973         2.2  154.5     NaN     NaN   
2         units          0  v41690973         2.2  155.3     NaN     NaN   
3         units          0  v41690973         2.2  156.4     NaN     NaN   
4         units          0  v41690973         2.2  157.0     NaN     NaN   

   TERMINATED  DECIMALS  Month  Year Month_Name  
0         NaN         1      1  2023        Jan  
1         NaN         1     

### 1. Seasonality — Best Month to Buy

In [26]:
## save the cleaned dataset to a csv file
dataset.to_csv("cleaned_dataset.csv", index=False)

In [15]:
# # Group by Product and Month
seasonality = dataset.groupby(['Product', 'Month']).agg(avg_value=('VALUE', 'mean')).reset_index()

# # Cheapest and most expensive month per product
cheapest = seasonality.loc[seasonality.groupby('Product')['avg_value'].idxmin()]
expensive = seasonality.loc[seasonality.groupby('Product')['avg_value'].idxmax()]

# # Merge results
seasonality_summary = pd.merge(
    cheapest[['Product', 'Month', 'avg_value']],
    expensive[['Product', 'Month', 'avg_value']],
    on='Product',
    suffixes=('_Cheapest', '_Expensive')
)

# # Show result
print(seasonality_summary.head())




                                             Product  Month_Cheapest  \
0  Alcoholic beverages, tobacco products and recr...               1   
1                                          All-items               1   
2                         All-items excluding energy               1   
3                All-items excluding food and energy               1   
4                              Clothing and footwear               1   

   avg_value_Cheapest  Month_Expensive  avg_value_Expensive  
0          191.033333                9           196.866667  
1          157.833333                8           161.766667  
2          154.533333                7           158.466667  
3          148.366667                7           151.933333  
4           92.600000                5            96.133333  


In [22]:
# # Save to CSV
seasonality_summary.to_csv("cpi_seasonality.csv", index=False)

### 2. Trap Index — Discount → Rebound

In [20]:
DROP_THRESHOLD = -0.02   # price drop ≥ 2%
REBOUND_THRESHOLD = 0.03 # rebound ≥ 3%
LOOKAHEAD = 2             # rebound within 2 months

trap_list = []

for product, group in dataset.groupby('Product'):
    group = group.sort_values('REF_DATE')
    group['pct_change'] = group['VALUE'].pct_change()
    
    dips = group[group['pct_change'] <= DROP_THRESHOLD].index
    
    for i in dips:
        for j in range(1, LOOKAHEAD + 1):
            if (i + j) in group.index:
                rebound = group.loc[i + j, 'pct_change']
                if rebound >= REBOUND_THRESHOLD:
                    trap_list.append({
                        'Product': product,
                        'Dip_Month': group.loc[i, 'REF_DATE'].strftime('%b'),
                        'Rebound_Month': group.loc[i + j, 'REF_DATE'].strftime('%b'),
                        'Dip_Pct': round(group.loc[i, 'pct_change'], 3),
                        'Rebound_Pct': round(rebound, 3)
                    })
                    break

trap_dataset = pd.DataFrame(trap_list)
print(trap_dataset.head())


    Product Dip_Month Rebound_Month  Dip_Pct  Rebound_Pct
0  Gasoline       Dec           Feb   -0.044         0.04


In [23]:
trap_dataset.to_csv("cpi_trap_index.csv", index=False)

### 3. Paycheck Pain — Hours Needed to Afford Food

In [21]:
NS_WAGE = 29.80  # Nova Scotia average hourly wage
BASE_COST = 100  # baseline food basket

pain_list = []

for product, group in dataset.groupby('Product'):
    group = group.sort_values('REF_DATE')
    base_value = group['VALUE'].iloc[0]
    
    group['price_proxy'] = BASE_COST * (group['VALUE'] / base_value)
    group['hours_needed'] = group['price_proxy'] / NS_WAGE
    
    pain_list.extend(group[['Product', 'REF_DATE', 'VALUE', 'price_proxy', 'hours_needed']].to_dict('records'))

pain_dataset = pd.DataFrame(pain_list)
print(pain_dataset.head())


                                             Product   REF_DATE  VALUE  \
0  Alcoholic beverages, tobacco products and recr... 2023-01-01  186.5   
1  Alcoholic beverages, tobacco products and recr... 2023-02-01  187.3   
2  Alcoholic beverages, tobacco products and recr... 2023-03-01  189.0   
3  Alcoholic beverages, tobacco products and recr... 2023-04-01  189.4   
4  Alcoholic beverages, tobacco products and recr... 2023-05-01  190.3   

   price_proxy  hours_needed  
0   100.000000      3.355705  
1   100.428954      3.370099  
2   101.340483      3.400687  
3   101.554960      3.407885  
4   102.037534      3.424078  


In [25]:
pain_dataset.to_csv("cpi_paycheck_pain.csv", index=False)